# Advanced Programming : Less Is More
 
*version 1, details/bugs in this notebook might be fixed within the first week of publication, you will be notified though studium if this happens.*

Every programming language has design trade-offs. Maybe the most prominent for python is that it is fast to code in, but not particularly efficient. We get around this when coding machine learning applications by utilizing modules with fast backends. This is why you should try to use NumPy instead of writing a linear algebra algorithm in pure python. But what about those cases where there are no fast backends, and we have to code more complex algorithms ourselves? This lab is role-playing such a case, where we will have to optimize our code (and algorithm implementation) in order to get enough of a speed-up to make the code usable.

The Mandelbrot zoom was a classic rite of initiation in some hacker circles in the 90s. We will do a version of this initiation rite, which is called the Julia fractal. The numbers in the Julia set are all complex numbers $z_0$, such that the series $z_{t+1} = z_t^2 + c$ is bounded (i.e. it does not go to infinity) given a $z_0$ in the interval $\{z \in \mathbb{C}: \operatorname{Re}(z) \in [-2,2], \; \operatorname{Im}(z) \in [-2,2]\}$. $c$ is a complex constant, which we will vary to create animation frames. We set the pixel colour in relation to the number of iterations $n$ in the series before it diverges, defined as $|z_n| > 2$. Note that you are given functioning code. Hence, you don't need to understand all of the maths. We will approach this as hackers, not mathematicians.

Pioneers in the field, like Gaston Julia and Benoit Mandelbrot, found these sets without using computers. For more on this, see:

* Barnsley, M. F. (2014). "*Fractals everywhere.*" Academic press.

* [Julia set](https://en.wikipedia.org/wiki/Julia_set)

For this lab, you are given code for generating a Julia fractal animation, however, the code is written in pure python and is really slow. Your task is as follows:
 
1. Copy the given code to the prepared cython code cell below.
2. Step by step, optimize the code using the tricks discussed in [Cython for NumPy users](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html).
3. The rendering of the frames is an *embarrassingly parallel* task. See to that your frames are rendered on several CPU cores.
4. Generate a 500 frame video at a 1000x1000 pixel resolution. To make it look nicer, the original resolution should be 2000x2000, resized to half size after applying the colourmap. The code for this step is given below.
 
You have passed when your code is **at least** 100 times faster than the given pure python implementation on an 8 core system. Good luck :)
 
## Submission
 
When you're done, hand in the notebook through studium. Requirements are:
1. Clean the notebook from unused code and unnecessary text.
2. Save the notebook with plotted images, but not the video (it’s too large).
3. Comment on what was hard and easy in the assignment (in a handful of sentences).

## Given implementation

Define a constant $c$.

In [ ]:
c = [complex(-0.835, -0.2321)]
c

The generator function.

In [ ]:
import numpy as np

def julia_v0(c, n_iter = 64, size = (500, 500)):
  """Julia set generator
  Generate Julia sets for multiple constants. The frames are returned as three
  dimensional matrix.
  
  c: list(complex)
    A list of complex numbers, used as the constant c for each Julia set frame.
  n_iter: int
    The maximum number of iterations when generateing the series.
  size: tuple(int, int)
    The size (height, width) of the output in pixels.

  return: numpy.array
    Numpy array as (frame, height, width), consisting of images of Julia sets.
  """
  # Store the pixels as lists of lists
  frames = list()
  # Loop over frames
  for frame in range(len(c)):
    # Create the frame entry
    frames.append(list())
    # Loop over rows
    for i in range(size[0]):
      frames[-1].append(list())
      # Loop over columns
      for j in range(size[1]):
        # Find the initial z, based on the position in the image
        z = complex(-2+j*4/size[0], 2-i*4/size[1])
        # 
        n = 0
        # Iterate to generate the series. Loop until a maximum number of 
        # allowed iterations or until the series diverges.
        while n < n_iter and abs(z) < 2: 
          # The update
          z = z*z + c[frame]
          # Increase the iteration counter
          n += 1
        # If the series did diverge, store the number of iterations it took (our
        # colouring will be proportional to this). If not, store a zero.
        if n == n_iter:
          frames[-1][-1].append(0)
        else:
          frames[-1][-1].append(n)
  # Return the frames to the user
  return np.asarray(frames)

Show one frame.

In [ ]:
import matplotlib.pyplot as plt   # Our standard plotting library

frames = julia_v0(c, n_iter=32, size=(500, 500))

plt.figure()
plt.imshow(frames[0], cmap='gray')
plt.axis('off')
plt.show()

## Your optimized implementation

### Use cython

We can save a lot of time by simply compiling our code.

The following loads the cython extension.

In [ ]:
%load_ext cython

The first line defines the rest of the cell as cython code. The python code will be translated into C code and compiled.

In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v1(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v1(c)[0], cmap='gray')
plt.axis('off')
plt.show()

#### Line profiler

A line profiler analyses each line of the code.

For this to work in cython, you need to specify the following compiler macros.

```
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1
```

But first, the line profiler needs some underlying software to work.

In [ ]:
!pip install line-profiler
%load_ext line_profiler

In [ ]:
%%cython -a
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
cimport cython

def julia_v1(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

To get an idea of where to start optimising, run the following cell and profile the given function. Try to find a critical inner loop and estimate the relative time spent there.

In [ ]:
%lprun -f julia_v1 julia_v1(c)

### Numpy array

Re-write the lists of lists data structure as a numpy array. When changing basic functionality, it can be a good idea to implement the changes in your pure python code first, then move the new functionality to your cython implementation. This is because cython gives less feedback when something goes wrong.

For more speed, look up the following decorators:
```
@cython.boundscheck(False)
@cython.wraparound(False)
```
For even more speed, look up contiguous arrays.


In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v2(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v2(c)[0], cmap='gray')
plt.axis('off')
plt.show()

### Type some variables

Try giving types to the indexing variables and the known integers. A surprisingly large amount of time is spent using these variables. Not because they do any heavy lifting, but simply because these pieces of code are run so many times.

In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v3(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v3(c)[0], cmap='gray')
plt.axis('off')
plt.show()

### Use doubles instead of python complex

When implementing optimization using cython, one might get carried away optimizing memory access or data structures (is this just me?), forgetting the core part of the algorithm. If you go back to the line profiler output, you can see that the inner loop (with the complex numbers) is where most of the CPU power is spent. As a complex number can be represented as two floating point numbers, change the inner loop to use floating point variables instead of python's `complex` data type.

If we parameterize the complex numbers z as (a, b), we can write the square $z_i = z^2_{i-1}$ as:

a*a

Try to find the smallest change you can do in your code, and test your changes thoroughly.

In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v4(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v4(c)[0], cmap='gray')
plt.axis('off')
plt.show()

### Look-up tables

Look-up tables can be used anywhere that you are repeating a caluclation. The initial values for $z_0$ are the same for each frame.

In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v5(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v5(c)[0], cmap='gray')
plt.axis('off')
plt.show()

### Multithreading

As this task is embarrassingly parallel, you can implement multi-threading using `prange`. You need some compiler directive to make this work, see below.

In [ ]:
%%cython -a

import numpy as np
cimport cython

def julia_v6(c, n_iter = 64, size = (500, 500)):
  raise NotImplementedError("This part is not finished yet.")

In [ ]:
plt.figure()
plt.imshow(julia_v6(c)[0], cmap='gray')
plt.axis('off')
plt.show()

## Compare execution times



In [ ]:
benchmarking_c = [complex(0.7885*np.cos(angle), 0.7885*np.cos(angle)) for angle in np.linspace(0, 2*np.pi, 10)]
max_iterations = 32
t = %timeit -o -n1 -r1 julia_v0(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking = [("Original", t.best)]
t = %timeit -o -n1 -r1 julia_v1(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("Cython", t.best))
t = %timeit -o -n1 -r1 julia_v2(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("Numpy array", t.best))
t = %timeit -o -n1 -r1 julia_v3(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("Basic typing", t.best))
t = %timeit -o julia_v4(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("Inner loop", t.best))
t = %timeit -o julia_v5(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("LUTs", t.best))
t = %timeit -o julia_v6(benchmarking_c, n_iter = max_iterations, size = (500, 500))
benchmarking.append(("Multithreading", t.best))

benchmarking

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.subplots(2, 1)
ax[0].bar(np.arange(len(benchmarking)), [time for name, time in benchmarking])
ax[0].set_xticks(np.arange(len(benchmarking)))
ax[0].set_xticklabels([name for name, time in benchmarking])
ax[0].set_ylabel("Time [seconds]")
speedup = benchmarking[0][1]/np.asarray([time for name, time in benchmarking])
h = ax[1].bar(np.arange(len(benchmarking)), speedup)
ax[1].set_xticks(np.arange(len(benchmarking)))
ax[1].set_xticklabels([name for name, time in benchmarking])
ax[1].set_ylabel("Speed-up")
# ax[1].set_yscale('log')
fig.tight_layout()
fig.show()

## Making a video

__*Don't run this until you are happy with your optimizations.*__

To make the frames nicer, a colourmap is applied to the raw counts of iterations. There are more beautiful colourmaps, but none in opencv. You can try out other colourmaps as described [here](https://docs.opencv.org/3.4.9/d3/d50/group__imgproc__colormap.html).

Below is a demo of the ones availible by default.

In [ ]:
import cv2

frames = julia_v6(c, n_iter = 64, size = (500, 500))
frame = np.asarray(255*frames[0]/frames[0].max(), dtype=np.uint8)
flags = [(flag, eval("cv2.%s" % flag)) for flag in dir(cv2) if flag[:8]=="COLORMAP"]

rows = 4
cols = (len(flags)+1)//rows
if rows*cols < len(flags):
  cols += 1

fig = plt.figure(figsize=(cols*3, rows*3), dpi=100)
ax = fig.subplots(rows, cols).ravel()
ax[0].imshow(frame, cmap='gray')
ax[0].set_title("Greyscale")
ax[0].axis('off')
for i, (label, cv2_colormap) in enumerate(flags):
  ax[i+1].imshow(cv2.cvtColor(cv2.applyColorMap(frame, cv2_colormap), cv2.COLOR_BGR2RGB))
  ax[i+1].set_title(label)
  ax[i+1].axis('off')
ax[i+2].imshow(cv2.bitwise_not(cv2.cvtColor(cv2.applyColorMap(frame, cv2.COLORMAP_BONE), cv2.COLOR_BGR2RGB)))
ax[i+2].set_title(label)
ax[i+2].axis('off')
fig.show()

In [ ]:
n_frames = 200
n_iterations = 64

import time
print("Generating frames...", end="")
t = time.time()
frames = julia_v6([complex(0.7885*np.cos(angle), 0.7885*np.sin(angle)) for angle in np.linspace(0, 2*np.pi, n_frames, endpoint=False)], 
                  n_iter = n_iterations, size = (2000, 2000))
print("done (%is)" % (time.time()-t))

This code is for viewing a sample of the generated images.

Note that opencv use BGR instead of RGB colour ordering.

In [ ]:
def process_image(frame, max_intensity=None):
  import cv2
  colormap = cv2.COLORMAP_HOT
  if max_intensity is None:
    image = np.asarray(255*frame/frame.max(), dtype=np.uint8)
  else:
    np.any(frame>max_intensity), "Make sure the maximum intensity in the fram is lower than max_intensity"
    image = np.asarray(255*frame/max_intensity, dtype=np.uint8)
  image = cv2.applyColorMap(image, colormap)
  image = cv2.bitwise_not(image) # Inverse colours
  image = cv2.resize(image, (image.shape[0]//2, image.shape[1]//2), interpolation=cv2.INTER_CUBIC)
  return image

fig = plt.figure(figsize=(5*3, 3), dpi=100)
ax = fig.subplots(1, 5).ravel()
for i, j in enumerate(np.linspace(0, frames.shape[0], 5, endpoint=False).astype(np.int)):
  ax[i].imshow(cv2.cvtColor(process_image(frames[j]), cv2.COLOR_BGR2RGB))
  ax[i].set_title("Frame %i" % j)
  ax[i].axis('off')
fig.show()

### MP4 video

This writes the video to disk. Note that if you do this through colab, your video will be written to the viritual machine's disk. You can find some code for downloading the file below.

In [ ]:
from tqdm import tqdm

# Prepare the writer class
first_video_frame = process_image(frames[0])
video = cv2.VideoWriter(filename='julia.mp4', 
                        fourcc = cv2.VideoWriter_fourcc(*'MP4V'), # video format
                        fps = 20, 
                        frameSize = (first_video_frame.shape[1], first_video_frame.shape[0]))

# Input the frames
for i in tqdm(range(frames.shape[0])):
  video.write(process_image(frames[i], max_intensity=64))

# Write to disk
video.release()

# Show the files in your working directory
!ls -l

### GIF animation

This creates a GIF animation from the frames with Julia sets. This method is a bit unreliable though and might cut frames sometimes. A simple fix is to use fewer frames in your animation.

In [ ]:
import os.path

if not os.path.exists("frames"):
  os.mkdir("frames")
else:
  files = ["frames/" + fn for fn in os.listdir("frames") if fn[-3:] == "png"]
  for file in files:
    os.remove(file)


for i in tqdm(range(frames.shape[0]), desc="Writing frames to disk"):
  cv2.imwrite("frames/frame%03i.png" % i, process_image(frames[i], max_intensity=64))

We will need imagemagick for this.

In [ ]:
!apt-get update
!apt-get install imagemagick

In [ ]:
# Studium course logo
# height: 146px;
# width: 262px;

!convert -resize 262x146 -delay 5 -loop 0 frames/frame*.png julia.gif

!ls -l